In [1]:
import pandas as pd
import datetime
from gsheets_connect import GoogleSheets, Connect

In [4]:
gsheet = GoogleSheets()

In [5]:
service = gsheet.service

In [6]:
def numberToLetters(q):
    q = q - 1
    result = ''
    while q >= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

In [7]:
cur_data = pd.read_csv(r'daily_data.txt', sep=' ')

#cur_data['Payment Day'] = pd.to_datetime(cur_data['Payment Day']).dt.date

cur_data['Payment Fortnight'] = cur_data['Payment Day'].apply(
    lambda date: str(pd.to_datetime(date).replace(day=15 if pd.to_datetime(date).day <= 15 else 30).date())
)

In [8]:
cur_data

,Amount $,Expense Day,Expense Month,Payment Day,Payment Month,Installments,Payment Method,Method Name,Category,Sub Category,Payment Fortnight
0,267.000000,2019-08-01,201908.0,2019-08-01,201908.0,1.0,debit,debit,Food,Restaurants,2019-08-15
1,633.000000,2019-08-12,201908.0,2019-08-15,201908.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-08-15
2,633.000000,2019-08-12,201908.0,2019-09-15,201909.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-09-15
3,633.000000,2019-08-12,201908.0,2019-10-15,201910.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-10-15
4,633.000000,2019-08-12,201908.0,2019-11-15,201911.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-11-15
5,633.000000,2019-08-12,201908.0,2019-12-15,201912.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-12-15
6,633.000000,2019-08-12,201908.0,2020-01-15,202001.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15
7,625.166667,2019-08-14,201908.0,2019-08-30,201908.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-08-30
8,625.166667,2019-08-14,201908.0,2019-09-30,201909.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-09-30
9,625.166667,2019-08-14,201908.0,2019-10-30,201910.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-10-30


In [9]:
cols = np.array(cur_data.columns)

cols_values = cols.tolist()

values_arr = cur_data.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [10]:
list_of_rows

array([['Amount $', 'Expense Day', 'Expense Month', 'Payment Day',
        'Payment Month', 'Installments', 'Payment Method', 'Method Name',
        'Category', 'Sub Category', 'Payment Fortnight'],
       [267.0, '2019-08-01', 201908.0, '2019-08-01', 201908.0, 1.0,
        'debit', 'debit', 'Food', 'Restaurants', '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-08-15', 201908.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-09-15', 201909.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-09-15'],
       [633.0, '2019-08-12', 201908.0, '2019-10-15', 201910.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-10-15'],
       [633.0, '2019-08-12', 201908.0, '2019-11-15', 201911.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-11-15'],
       [633.0, '2019-08-1

In [80]:
categories_titles = gsheet.gsheet_to_df(
    spreadsheet_id = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4',
    range_name = 'biweekly plan!B27:D100'
)

In [81]:
separations = categories_titles[
    categories_titles['Variable Expenses'].isnull()
].index

In [82]:
separations#[:-1]

Int64Index([0, 7, 12, 18, 24, 32, 37], dtype='int64')

In [95]:
cats = {}
sub_cats = {}
for i in range(1,len(separations)):
    ix_1 = separations[i-1]+1
    ix_2 = separations[i]-1
    cat = categories_titles.loc[ix_1:ix_2, 'Variable Expenses']
    cats[i] =  cat.iloc[0]
    sc = {}
    for j, c in enumerate(cat.iloc[1:]):
        
        sc[j+1] = c
    sub_cats[cat.iloc[0]] = sc
cat = categories_titles.loc[separations[-1]+1:, 'Variable Expenses']
cats[len(separations)] =  cat.iloc[0]

In [96]:
cats

{1: 'Food',
 2: 'Getting Around',
 3: 'Fun Stuff',
 4: 'Health Care',
 5: 'Personal Stuff',
 6: 'Apartment Spends',
 7: 'Removed From Savings'}

In [97]:
sub_cats

{'Food': {1: 'Super Market',
  2: 'Restaurants',
  3: 'Take-out and bingeing',
  4: 'Coffe',
  5: 'Other'},
 'Getting Around': {1: 'Gas', 2: 'Parking', 3: 'Other'},
 'Fun Stuff': {1: 'Socializing and Bars',
  2: 'Hobbies',
  3: 'Books',
  4: 'Other'},
 'Health Care': {1: 'Medicine', 2: 'Vitamins', 3: 'Supplements', 4: 'Other'},
 'Personal Stuff': {1: 'Clothing',
  2: 'Gifts',
  3: 'Haircut',
  4: 'Stuff for me',
  5: 'Personal Care',
  6: 'Other'},
 'Apartment Spends': {1: 'Stuff for the Apartment', 2: 'Services', 3: 'Other'}}

In [45]:
main_categories = categories_titles.loc[
    categories_titles[
        categories_titles['Variable Expenses'].isnull()
    ].index + 1
]['Variable Expenses']

In [53]:
main_categories.index

Int64Index([1, 8, 13, 19, 25, 33], dtype='int64')

In [55]:
sub_categories = categories_titles.loc[
    set(categories_titles.index) - set(main_categories.index) - set(separations)
]

In [56]:
sub_categories

,Variable Expenses,"8,670","8,670"
2,Super Market,"1,000","1,000"
3,Restaurants,700,700
4,Take-out and bingeing,120,120
5,Coffe,120,120
6,Other,60,60
9,Gas,800,800
10,Parking,100,100
11,Other,100,100
14,Socializing and Bars,800,800
15,Hobbies,500,500


In [46]:
categories_titles

,Variable Expenses,"8,670","8,670"
0,NaN,NaN,NaN
1,Food,"2,000","2,000"
2,Super Market,"1,000","1,000"
3,Restaurants,700,700
4,Take-out and bingeing,120,120
5,Coffe,120,120
6,Other,60,60
7,NaN,NaN,NaN
8,Getting Around,"1,000","1,000"
9,Gas,800,800


In [11]:
data = [
    {
        'range': 'data!A1',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

154 cells updated.


In [12]:
monthly_data=cur_data.groupby(
    ['Payment Fortnight','Category', 'Sub Category']
).agg({'Amount $':'sum'})

In [14]:
pd.pivot_table(
    cur_data,
    index='Payment Fortnight',
    columns = ['Category', 'Sub Category'],
    values = ['Amount $']
)

Amount $               
Category                 Food Personal Stuff
Sub Category      Restaurants   Stuff for me
Payment Fortnight                           
2019-08-15              267.0     633.000000
2019-08-30                NaN     625.166667
2019-09-15                NaN     633.000000
2019-09-30                NaN     625.166667
2019-10-15                NaN     633.000000
2019-10-30                NaN     625.166667
2019-11-15                NaN     633.000000
2019-11-30                NaN     625.166667
2019-12-15                NaN     633.000000
2019-12-30                NaN     625.166667
2020-01-15                NaN     633.000000
2020-01-30                NaN     625.166667

In [13]:
monthly_data#.unstack(0)

Amount $
Payment Fortnight Category       Sub Category            
2019-08-15        Food           Restaurants   267.000000
                  Personal Stuff Stuff for me  633.000000
2019-08-30        Personal Stuff Stuff for me  625.166667
2019-09-15        Personal Stuff Stuff for me  633.000000
2019-09-30        Personal Stuff Stuff for me  625.166667
2019-10-15        Personal Stuff Stuff for me  633.000000
2019-10-30        Personal Stuff Stuff for me  625.166667
2019-11-15        Personal Stuff Stuff for me  633.000000
2019-11-30        Personal Stuff Stuff for me  625.166667
2019-12-15        Personal Stuff Stuff for me  633.000000
2019-12-30        Personal Stuff Stuff for me  625.166667
2020-01-15        Personal Stuff Stuff for me  633.000000
2020-01-30        Personal Stuff Stuff for me  625.166667

In [55]:
monthly_data.unstack(0).shape

(2, 12)

In [58]:
numberToLetters(12)

'L'

In [ ]:
rows_range = 

In [39]:
dates = np.array(monthly_data.unstack(0).columns.levels[1])

cols_values = np.array(['category', 'sub_category']+dates.astype(str).tolist())

values_arr = monthly_data.unstack(0).fillna(0).reset_index().values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [59]:
list_of_rows.shape

(3, 14)

In [71]:
start_from = 'biweekly pivot!B2'

In [66]:
table_range

'B2:N3'

In [72]:
data = [
    {
        'range': start_from,
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

42 cells updated.


In [9]:
list_of_rows

[['Food',
  'Restaurants',
  267.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 ['Personal Stuff',
  'Stuff for me',
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666]]